# Simulate Toy GRN to benchmark velocity, latent time and GRN inference performance

## Library imports

In [1]:
import torch
from torch import nn
import torchode as to

import regvelovi
from regvelovi import REGVELOVI
from typing import Literal
from velovi import preprocess_data, VELOVI
import numpy as np
import pandas as pd
import sklearn
import anndata
import anndata as ad

import scvelo as scv
import scipy
import seaborn as sns
import scanpy as sc
from paths import FIG_DIR, DATA_DIR

[rank: 0] Global seed set to 0


In [2]:
from arboreto.algo import grnboost2, genie3
from arboreto.utils import load_tf_names

In [3]:
import celloracle as co

In [4]:
import os

## General settings

In [5]:
sns.reset_defaults()
sns.reset_orig()
scv.settings.set_figure_params('scvelo', dpi_save=400, dpi=80, transparent=True, fontsize=20, color_map='viridis')

In [6]:
SAVE_FIGURES = True
if SAVE_FIGURES:
    os.makedirs(FIG_DIR / 'simulation' / 'toy_GRN',exist_ok = True)

SAVE_DATASETS = True
if SAVE_DATASETS:
    os.makedirs(DATA_DIR / 'simulation' / 'toy_GRN',exist_ok = True)

## Function defination

In [7]:
def sign_concordance(GRN,ref_GRN):
    sign_GRN = np.sign(GRN)[GRN!=0]
    sign_ref_GRN = np.sign(ref_GRN)[GRN!=0]
    score = sum(sign_GRN == sign_ref_GRN)
    return score
    
def calculate_power_matrix(A, B):
    if len(A) != len(B) or len(A[0]) != len(B[0]):
        raise ValueError("Both matrices must have the same dimensions")

    rows = len(A)
    cols = len(A[0])

    C = [[0 for _ in range(cols)] for _ in range(rows)]

    for i in range(rows):
        for j in range(cols):
            C[i][j] = A[i][j] ** B[i][j]

    return np.array(C)
    
class velocity_encoder(nn.Module):
    """ 
    encode the velocity
    time dependent transcription rate is determined by upstream emulator
    velocity could be build on top of this

    merge velocity encoder and emulator class
    """                 
    def __init__(
        self,
        K,
        n,
        h,
        alpha_b,
        beta,
        gamma,
    ):
        super().__init__()
        self.K = K
        self.n = n
        self.h = h
        self.alpha_b = alpha_b
        self.beta = beta
        self.gamma = gamma

    def forward(self,t,y):
        """
        s: spliced readout of the genes
        K: maximum contribution of regulator j to target gene i,
        n: the Hill coefficient that introduces non-linearity to the model
        h: the regulator concentration that produces half-maximal regulatory effect (half response)
        """
        u = y[:,0].ravel()
        s = y[:,1].ravel()
        
        sign = torch.sign(self.K)
        sign = torch.clip(torch.sign(self.K),0,1)
        
        s_m = s.repeat(self.n.shape[0],1)
        x_n = torch.pow(torch.clip(s_m,0,), self.n)
        h_n = self.h**self.n
        
        p_act = (x_n/(h_n+x_n))
        p_neg = (h_n/(h_n+x_n))

        """
        print(p_act)
        print(p_neg)
        """
        
        p = torch.abs(self.K)*((p_act*sign)+(p_neg*(1-sign)))
        alpha = p.sum(1) + self.alpha_b
        
        du = torch.clip(alpha,0,) - self.beta*u + torch.randn(len(u)) ## add noise -> SDE
        ds = self.beta*u - self.gamma*s + torch.randn(len(s)) 

        du = du.reshape((-1,1))
        ds = ds.reshape((-1,1))
        
        v = torch.concatenate([du,ds],axis = 1)
        #v = du.reshape((-1,1))

        return v

In [8]:
def add_regvelo_outputs_to_adata(adata_raw, vae, filter = False):
    # This code is adapted on an implementation by Adam Gayoso, Philipp Weilier and Justing Hong from their repository:
    # https://github.com/YosefLab/velovi_reproducibility
    # Original function: fit_velovi() in https://github.com/YosefLab/velovi_reproducibility/blob/main/notebooks/estimation_comparison/fucci.ipynb
    # License: BSD-3-Clause license

    latent_time = vae.get_latent_time(n_samples=30, time_statistic = "mean",batch_size = adata_raw.shape[0])
    velocities = vae.get_velocity(n_samples=30, velo_statistic="mean",batch_size = adata_raw.shape[0])

    t = latent_time
    scaling = 20 / t.max(0)
    adata = adata_raw[:,vae.module.target_index].copy()

    adata.layers["velocity"] = velocities
    adata.layers["latent_time_regvelo"] = latent_time

    adata.layers["fit_t"] = latent_time.values * np.array(scaling)[np.newaxis, :]
    adata.var['fit_scaling'] = 1.0
    adata.var['fit_beta_regvelo'] = torch.clip(torch.nn.functional.softplus(reg_vae.module.v_encoder.beta_mean_unconstr),0,50).cpu().detach().numpy() / scaling
    adata.var['fit_gamma_regvelo'] = torch.clip(torch.nn.functional.softplus(reg_vae.module.v_encoder.gamma_mean_unconstr),0,50).cpu().detach().numpy() / scaling

    return adata

def add_velovi_outputs_to_adata(adata, vae):
    # This code is adapted on an implementation by Adam Gayoso, Philipp Weilier and Justing Hong from their repository:
    # https://github.com/YosefLab/velovi_reproducibility
    # Original function: fit_velovi() in https://github.com/YosefLab/velovi_reproducibility/blob/main/notebooks/estimation_comparison/fucci.ipynb
    # License: BSD-3-Clause license
    
    latent_time = vae.get_latent_time(n_samples=30)
    velocities = vae.get_velocity(n_samples=30, velo_statistic="mean")

    t = latent_time
    scaling = 20 / t.max(0)

    adata.layers["velocity"] = velocities / scaling
    adata.layers["latent_time_velovi"] = latent_time

    adata.var["fit_alpha_velovi"] = vae.get_rates()["alpha"] / scaling
    adata.var["fit_beta_velovi"] = vae.get_rates()["beta"] / scaling
    adata.var["fit_gamma_velovi"] = vae.get_rates()["gamma"] / scaling
    adata.var["fit_t_"] = (
        torch.nn.functional.softplus(vae.module.switch_time_unconstr)
        .detach()
        .cpu()
        .numpy()
    ) * scaling
    adata.layers["fit_t"] = latent_time.values * np.array(scaling)[np.newaxis, :]
    adata.var['fit_scaling'] = 1.0

## Initialize all values

In [9]:
velo_scv = []
velo_velovi = []
velo_rgv = []
corr_t_rgv = []
corr_t_scv = []
corr_t_velovi = []
corr_t_emperical = []
corr_t_dpt = []
beta_list = []
gamma_list = []
rgv_GRN = []
cor_GRN = []
gb_GRN = []
co_GRN = []

## Run simulation for 100 times

In [10]:
for sim_idx in range(100):
    print(sim_idx)
    torch.cuda.empty_cache()
    torch.manual_seed(sim_idx)


    ## simulate alpha beta and gamma
    n_vars = 6

    mu = np.array([np.log(5),np.log(.5), np.log(.125)])
    
    R = np.array([[1., .2, .2],
                  [.2, 1., .8],
                  [.2, .8, 1.]])
    
    C = np.array([.4, .4, .4])[:, None]
    
    cov = C.dot(C.T) * R
    
    alpha, beta, gamma = np.exp(np.random.multivariate_normal(mu, cov, size=n_vars).T)  # multivariate log-normal
    #beta /= 3
    #gamma /= 10
    beta_list.append(beta)
    gamma_list.append(gamma)
    
    ## 
    n_regulators = n_targets = 6
    coef_m = np.array([[0,1,-alpha.mean(),2,2],
                       [1,0,-alpha.mean(),2,2],
                       [0,2,alpha.mean(),2,4],
                       [0,3,alpha.mean(),2,4],
                       [2,3,-alpha.mean(),2,2],
                       [3,2,-alpha.mean(),2,2],
                       [1,4,alpha.mean(),2,4],
                       [1,5,alpha.mean(),2,4],
                       [4,5,-alpha.mean(),2,2],
                       [5,4,-alpha.mean(),2,2]])
    
    random_seed = sim_idx
    def draw_poisson(n):
        from random import seed, uniform  # draw from poisson
        seed(random_seed)
        t = np.cumsum([-0.1 * np.log(uniform(0, 1)) for _ in range(n - 1)])
        return np.insert(t, 0, 0)  # prepend t0=0
        
    t = torch.tensor(draw_poisson(1500))
    t = t.repeat(n_targets,1)

    K = np.zeros([n_targets, n_regulators])
    n = np.zeros([n_targets, n_regulators])
    h = np.zeros([n_targets, n_regulators])

    K[np.array(coef_m[:,1],dtype = int),np.array(coef_m[:,0],dtype = int)] = coef_m[:,2]
    n[np.array(coef_m[:,1],dtype = int),np.array(coef_m[:,0],dtype = int)] = coef_m[:,3]
    h[np.array(coef_m[:,1],dtype = int),np.array(coef_m[:,0],dtype = int)] = coef_m[:,4]

    v_encoder = velocity_encoder(K=torch.tensor(K, dtype=torch.float32),
                                 n=torch.tensor(n, dtype=torch.float32),
                                 h=torch.tensor(h, dtype=torch.float32),
                                 alpha_b=torch.tensor([1, 1, 1, 1, 1, 1], dtype=torch.float32) * 0,
                                 beta=torch.tensor(beta,dtype = torch.float32),
                                 gamma=torch.tensor(gamma,dtype = torch.float32))

    ## set up G batches, Each G represent a module (a target gene centerred regulon)
    ## infer the observe gene expression through ODE solver based on x0, t, and velocity_encoder
    term = to.ODETerm(v_encoder)
    step_method = to.Dopri5(term=term)
    #step_size_controller = to.IntegralController(atol=1e-6, rtol=1e-3, term=term)
    step_size_controller = to.FixedStepController()
    solver = to.AutoDiffAdjoint(step_method, step_size_controller)
    #jit_solver = torch.jit.script(solver)
    sol = solver.solve(to.InitialValueProblem(y0=torch.tensor([[1.0,0,1.0,0,1.0,0],torch.zeros(6).abs().tolist()]).T, t_eval=t), dt0 = 0.01*torch.ones(K.shape[0]))

    pre_u = sol.ys[:,:,0]
    pre_s = sol.ys[:,:,1]

    pre_s = pd.DataFrame(pre_s.T.numpy())
    pre_u = pd.DataFrame(pre_u.T.numpy())
    adata = ad.AnnData(np.array(pre_s))
    
    
    ## Preprocessing
    adata.layers["Ms"] = np.array(pre_s)
    adata.layers["Mu"] = np.array(pre_u)
    adata.layers["spliced"] = np.array(pre_s)
    adata.layers["unspliced"] = np.array(pre_u)
    adata.obs_names = [f"Cell_{i:d}" for i in range(adata.n_obs)]
    adata.var_names = [f"Gene_{i:d}" for i in range(adata.n_vars)]

    adata.obs["time"] = t.numpy()[0,:]
    reg_bdata = adata.copy()
    reg_bdata.uns["regulators"] = adata.var.index.values
    reg_bdata.uns["targets"] = adata.var.index.values
    reg_bdata.uns["skeleton"] = np.ones((len(adata.var.index),len(adata.var.index)))
    reg_bdata.uns["network"] = np.ones((len(adata.var.index),len(adata.var.index)))

    ## veloVI & RegVelo min-max scaling
    reg_bdata = preprocess_data(reg_bdata,filter_on_r2 = False)

    ## delete self-regulation
    W = reg_bdata.uns["skeleton"].copy()
    W = torch.tensor(np.array(W)).int()
        
    REGVELOVI.setup_anndata(reg_bdata, spliced_layer="Ms", unspliced_layer="Mu")
    reg_vae = REGVELOVI(reg_bdata,W=W,lam2 = 1,soft_constraint = False,simple_dynamics = True)
    reg_vae.train()
    
    adata_target = add_regvelo_outputs_to_adata(reg_bdata, reg_vae)
    pre_t = adata_target.layers["latent_time_regvelo"].mean(1)
    pre_t = t.max() * ((pre_t - np.min(pre_t)) / (np.max(pre_t) - np.min(pre_t)))
    adata_target.obs["latent_time_regvelo"] = pre_t
    
    ## print regvelo performance
    corr_t_rgv.append(scipy.stats.spearmanr(pre_t,adata.obs["time"])[0])
    velo_rgv.append(adata_target.layers["velocity"])
    print("RegVelo: "+str(scipy.stats.spearmanr(pre_t,adata.obs["time"])[0]))

    ## Run scVelo
    sc.pp.neighbors(reg_bdata)
    scv.tl.recover_dynamics(
        reg_bdata, fit_scaling=False, var_names=adata.var_names, n_jobs=1
    )
    reg_bdata.var["fit_scaling"] = 1.0
    scv.tl.velocity(reg_bdata, mode="dynamical", min_likelihood=-np.inf, min_r2=None)
    pre_t = reg_bdata.layers["fit_t"].mean(1)
    pre_t = t.max() * ((pre_t - np.min(pre_t)) / (np.max(pre_t) - np.min(pre_t)))
    
    ## print regvelo performance
    corr_t_scv.append(scipy.stats.spearmanr(pre_t,adata.obs["time"])[0])
    velo_scv.append(reg_bdata.layers["velocity"])
    print("scVelo: "+str(scipy.stats.spearmanr(pre_t,adata.obs["time"])[0]))
    
    ## import veloVI
    VELOVI.setup_anndata(adata_target, spliced_layer="Ms", unspliced_layer="Mu")
    vae = VELOVI(adata_target)
    vae.train(max_epochs = 500,early_stopping = True)
    add_velovi_outputs_to_adata(adata_target, vae)
    pre_t = adata_target.layers["fit_t"].mean(1)
    pre_t = 200 * ((pre_t - np.min(pre_t)) / (np.max(pre_t) - np.min(pre_t)))
    
    corr_t_velovi.append(scipy.stats.spearmanr(pre_t,adata.obs["time"])[0])
    velo_scv.append(adata_target.layers["velocity"])
    print("veloVI: "+str(scipy.stats.spearmanr(pre_t,adata.obs["time"])))
    
    adata_target.obs["latent_time_velovi"] = pre_t
    ### calculate diffusion pseudotime
    adata_target.uns['iroot'] = np.flatnonzero(adata_target.obs['time']  == 0)[0]
    
    sc.pp.neighbors(adata_target)
    sc.tl.diffmap(adata_target)
    sc.tl.dpt(adata_target)

    adata_target.obs["emperical_time"] = adata_target.layers["Mu"].mean(1)
    corr_t_emperical.append(scipy.stats.spearmanr(adata_target.obs["emperical_time"],adata.obs["time"])[0])
    corr_t_dpt.append(scipy.stats.spearmanr(adata_target.obs["dpt_pseudotime"],adata.obs["time"])[0])

    #### Benchmark GRN performance
    GRN = reg_vae.module.v_encoder.GRN_Jacobian(torch.tensor(np.array(pre_s)).mean(0).to("cuda:0")).cpu().detach().cpu().numpy()
    pre = GRN[np.where(~np.eye(GRN.shape[0],dtype=bool))]
    label = K[np.where(~np.eye(K.shape[0],dtype=bool))]
    label[label != 0] = 1
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(label,pre, pos_label=1)
    rgv_GRN.append(sklearn.metrics.auc(fpr, tpr))

    # calculate correlation
    C = np.abs(np.array(pd.DataFrame(adata.layers["spliced"]).corr()))
    pre2 = C[np.where(~np.eye(C.shape[0],dtype=bool))]
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(label,pre2, pos_label=1)
    cor_GRN.append(sklearn.metrics.auc(fpr, tpr))
    
    # GRNBoost2
    ex_matrix = adata.to_df("spliced")
    tf = ex_matrix.columns.tolist()
    network = grnboost2(expression_data=ex_matrix,
                    tf_names=tf)
    table = np.array(network)

    # Get unique TFs and targets
    unique_tfs = np.unique(table[:, 0])
    unique_targets = np.unique(table[:, 1])
    
    # Create a new NumPy array to store the rearranged data
    GRN = np.zeros((len(unique_targets), len(unique_tfs)))
    
    # Fill in the new array with importance values
    for row in table:
        tf_index = np.where(unique_tfs == row[0])[0][0]
        target_index = np.where(unique_targets == row[1])[0][0]
        GRN[target_index, tf_index] = row[2]

    pre = GRN[np.where(~np.eye(GRN.shape[0],dtype=bool))]
    label = K[np.where(~np.eye(K.shape[0],dtype=bool))]
    label[label != 0] = 1
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(label,pre, pos_label=1)
    gb_GRN.append(sklearn.metrics.auc(fpr, tpr))

    ## Run celloracle
    base_GRN_sim = np.array(W)
    base_GRN_sim[base_GRN_sim!=0] = 1
    base_GRN_sim = pd.DataFrame(base_GRN_sim,columns = ["Gene0","Gene1","Gene2","Gene3","Gene4","Gene5"])
    base_GRN_sim.loc[:,"peak_id"] = [(f'Peak_{i}') for i in ["0","1","2","3","4","5"]].copy()
    base_GRN_sim.loc[:,"gene_short_name"] = ["Gene0","Gene1","Gene2","Gene3","Gene4","Gene5"]
    base_GRN_sim = base_GRN_sim.loc[:,["peak_id","gene_short_name","Gene0","Gene1","Gene2","Gene3","Gene4","Gene5"]]

    adata.var.index = ["Gene0","Gene1","Gene2","Gene3","Gene4","Gene5"]
    net = co.Net(gene_expression_matrix=adata.to_df(), # Input gene expression matrix as data frame
             TFinfo_matrix=base_GRN_sim, # Input base GRN
             verbose=True
             )

    net.fit_All_genes(bagging_number=100,
                 alpha=1, verbose=True)
    net.updateLinkList(verbose=True)
    inference_result = net.linkList.copy()

    GRN_table = inference_result.iloc[:,:3].copy()
    table = np.array(GRN_table)

    # Get unique TFs and targets
    unique_tfs = np.unique(table[:, 0])
    unique_targets = np.unique(table[:, 1])
    
    # Create a new NumPy array to store the rearranged data
    GRN = np.zeros((len(unique_targets), len(unique_tfs)))
    
    # Fill in the new array with importance values
    for row in table:
        tf_index = np.where(unique_tfs == row[0])[0][0]
        target_index = np.where(unique_targets == row[1])[0][0]
        GRN[target_index, tf_index] = row[2]
    pre = np.abs(GRN)[np.where(~np.eye(GRN.shape[0],dtype=bool))]
    label = K[np.where(~np.eye(K.shape[0],dtype=bool))]
    label[label != 0] = 1
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(label,pre, pos_label=1)
    co_GRN.append(sklearn.metrics.auc(fpr, tpr))

    print("AUC: "+"RegVelo: "+str(rgv_GRN[len(rgv_GRN)-1])+" Cor: "+str(cor_GRN[len(cor_GRN)-1])+" GRNBoost2: "+str(gb_GRN[len(gb_GRN)-1])+" CellOracle: "+str(co_GRN[len(co_GRN)-1]))

0


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 400/1500:  27%|██▋       | 400/1500 [01:05<02:59,  6.11it/s, loss=-8.89, v_num=1]  
Monitored metric elbo_validation did not improve in the last 45 records. Best score: -7.549. Signaling Trainer to stop.
RegVelo: 0.8775575135811173
recovering dynamics (using 1/112 cores)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


    finished (0:00:00) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)
computing velocities
    finished (0:00:00) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
scVelo: 0.5727277065383075
Epoch 318/500:  64%|██████▎   | 318/500 [00:28<00:16, 11.27it/s, loss=-7.26, v_num=1]
Monitored metric elbo_validation did not improve in the last 45 records. Best score: -7.040. Signaling Trainer to stop.
veloVI: SignificanceResult(statistic=0.7841196551642912, pvalue=1.152714405e-312)
initiating Net object ...
gem_shape: (1500, 6)
TF info shape: (6, 8)
initiation completed.


...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


    finished (0:00:00) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)
computing velocities
    finished (0:00:00) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
scVelo: 0.6215007862054314
Epoch 302/500:  60%|██████    | 302/500 [00:30<00:20,  9.86it/s, loss=-5.08, v_num=1]
Monitored metric elbo_validation did not improve in the last 45 records. Best score: -5.241. Signaling Trainer to stop.
veloVI: SignificanceResult(statistic=0.5760324454829223, pvalue=2.5206810975198528e-133)
initiating Net object ...
gem_shape: (1500, 6)
TF info shape: (6, 8)
initiation completed.


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

AUC: RegVelo: 0.71 Cor: 0.61 GRNBoost2: 0.62 CellOracle: 0.47500000000000003


In [11]:
## boxplot to show latent time correlation on each gene
dat = pd.DataFrame({"Time":  corr_t_rgv + corr_t_velovi  + corr_t_scv + corr_t_dpt,
"Model": ["RegVelo"]*100 + ["veloVI"]*100 + ["scVelo"]*100 + ["Diffusion pseudotime"]*100})
if SAVE_DATASETS:
    dat.to_csv(DATA_DIR / 'simulation' / 'toy_GRN' / 'latent_time_benchmark_result.csv')

In [12]:
dat = pd.DataFrame({"GRN":  rgv_GRN + cor_GRN + gb_GRN + co_GRN,
"Model": ["RegVelo"]*100 + ["Correlation"]*100 + ["GRNBoost2"]*100 + ["CellOracle"]*100})
if SAVE_DATASETS:
    dat.to_csv(DATA_DIR / 'simulation' / 'toy_GRN' / 'GRN_benchmark_result.csv')

## Benchmark velocity inference

In [10]:
## Benchmark velocity inference
velo_scv = []
velo_velovi = []
velo_rgv = []

In [11]:
for sim_idx in range(100):
    print(sim_idx)
    torch.cuda.empty_cache()
    torch.manual_seed(sim_idx)


    ## simulate alpha beta and gamma
    n_vars = 6

    mu = np.array([np.log(5),np.log(.5), np.log(.125)])
    
    R = np.array([[1., .2, .2],
                  [.2, 1., .8],
                  [.2, .8, 1.]])
    
    C = np.array([.4, .4, .4])[:, None]
    
    cov = C.dot(C.T) * R
    
    alpha, beta, gamma = np.exp(np.random.multivariate_normal(mu, cov, size=n_vars).T)  # multivariate log-normal
    #beta /= 3
    #gamma /= 10
    beta_list.append(beta)
    gamma_list.append(gamma)
    
    ## 
    n_regulators = n_targets = 6
    coef_m = np.array([[0,1,-alpha.mean(),2,2],
                       [1,0,-alpha.mean(),2,2],
                       [0,2,alpha.mean(),2,4],
                       [0,3,alpha.mean(),2,4],
                       [2,3,-alpha.mean(),2,2],
                       [3,2,-alpha.mean(),2,2],
                       [1,4,alpha.mean(),2,4],
                       [1,5,alpha.mean(),2,4],
                       [4,5,-alpha.mean(),2,2],
                       [5,4,-alpha.mean(),2,2]])
    
    random_seed = sim_idx
    def draw_poisson(n):
        from random import seed, uniform  # draw from poisson
        seed(random_seed)
        t = np.cumsum([-0.1 * np.log(uniform(0, 1)) for _ in range(n - 1)])
        return np.insert(t, 0, 0)  # prepend t0=0
        
    t = torch.tensor(draw_poisson(1500))
    t = t.repeat(n_targets,1)

    K = np.zeros([n_targets, n_regulators])
    n = np.zeros([n_targets, n_regulators])
    h = np.zeros([n_targets, n_regulators])

    K[np.array(coef_m[:,1],dtype = int),np.array(coef_m[:,0],dtype = int)] = coef_m[:,2]
    n[np.array(coef_m[:,1],dtype = int),np.array(coef_m[:,0],dtype = int)] = coef_m[:,3]
    h[np.array(coef_m[:,1],dtype = int),np.array(coef_m[:,0],dtype = int)] = coef_m[:,4]

    v_encoder = velocity_encoder(K=torch.tensor(K, dtype=torch.float32),
                                 n=torch.tensor(n, dtype=torch.float32),
                                 h=torch.tensor(h, dtype=torch.float32),
                                 alpha_b=torch.tensor([1, 1, 1, 1, 1, 1], dtype=torch.float32) * 0,
                                 beta=torch.tensor(beta,dtype = torch.float32),
                                 gamma=torch.tensor(gamma,dtype = torch.float32))

    ## set up G batches, Each G represent a module (a target gene centerred regulon)
    ## infer the observe gene expression through ODE solver based on x0, t, and velocity_encoder
    term = to.ODETerm(v_encoder)
    step_method = to.Dopri5(term=term)
    #step_size_controller = to.IntegralController(atol=1e-6, rtol=1e-3, term=term)
    step_size_controller = to.FixedStepController()
    solver = to.AutoDiffAdjoint(step_method, step_size_controller)
    #jit_solver = torch.jit.script(solver)
    sol = solver.solve(to.InitialValueProblem(y0=torch.tensor([[1.0,0,1.0,0,1.0,0],torch.zeros(6).abs().tolist()]).T, t_eval=t), dt0 = 0.01*torch.ones(K.shape[0]))

    pre_u = sol.ys[:,:,0]
    pre_s = sol.ys[:,:,1]
        
    pre_s = pd.DataFrame(pre_s.T.numpy())
    pre_u = pd.DataFrame(pre_u.T.numpy())
    gt_velo = np.array(pre_u)*beta - np.array(pre_s)*gamma
    adata = ad.AnnData(np.array(pre_s))

    adata.layers["Ms"] = np.array(pre_s)
    adata.layers["Mu"] = np.array(pre_u)
    adata.layers["spliced"] = np.array(pre_s)
    adata.layers["unspliced"] = np.array(pre_u)
    adata.obs_names = [f"Cell_{i:d}" for i in range(adata.n_obs)]
    adata.var_names = [f"Gene_{i:d}" for i in range(adata.n_vars)]

    adata.obs["time"] = t.numpy()[0,:]
    reg_bdata = adata.copy()
    reg_bdata.uns["regulators"] = adata.var.index.values
    reg_bdata.uns["targets"] = adata.var.index.values
    reg_bdata.uns["skeleton"] = np.ones((len(adata.var.index),len(adata.var.index)))
    reg_bdata.uns["network"] = np.ones((len(adata.var.index),len(adata.var.index)))

    
    ## veloVI & RegVelo min-max scaling
    reg_bdata = preprocess_data(reg_bdata,filter_on_r2 = False)

    ## delete self-regulation
    W = reg_bdata.uns["skeleton"].copy()
    W = torch.tensor(np.array(W)).int()
    
    REGVELOVI.setup_anndata(reg_bdata, spliced_layer="Ms", unspliced_layer="Mu")
    reg_vae = REGVELOVI(reg_bdata,W=W,lam2 = 0.5,soft_constraint = False,simple_dynamics = True)
    reg_vae.train()
    
    adata_target = add_regvelo_outputs_to_adata(reg_bdata, reg_vae)
    pre_velo = np.array(pre_u)*np.array(adata_target.var["fit_beta_regvelo"]) - np.array(pre_s)*np.array(adata_target.var["fit_gamma_regvelo"])
    corr_rgv = []
    for i in range(6):
        corr_rgv.append(scipy.stats.pearsonr(pre_velo[:,i],gt_velo[:,i])[0])
    
    ## print regvelo performance
    velo_rgv.append(np.mean(corr_rgv))
    print("RegVelo: "+str(np.mean(corr_rgv)))


    ## Run scVelo
    sc.pp.neighbors(reg_bdata)
    scv.tl.recover_dynamics(
        reg_bdata, fit_scaling=False, var_names=adata.var_names, n_jobs=1
    )
    reg_bdata.var["fit_scaling"] = 1.0
    scv.tl.velocity(reg_bdata, mode="dynamical", min_likelihood=-np.inf, min_r2=None)

    pre_velo = np.array(pre_u)*np.array(reg_bdata.var["fit_beta"]) - np.array(pre_s)*np.array(reg_bdata.var["fit_gamma"])
    corr_scv = []
    for i in range(6):
        corr_scv.append(scipy.stats.pearsonr(pre_velo[:,i],gt_velo[:,i])[0])
    
    ## print regvelo performance
    velo_scv.append(np.mean(corr_scv))
    print("veloVI: "+str(np.mean(corr_scv)))
    
    ## import veloVI
    VELOVI.setup_anndata(adata_target, spliced_layer="Ms", unspliced_layer="Mu")
    vae = VELOVI(adata_target)
    vae.train(max_epochs = 500,early_stopping = True)

    add_velovi_outputs_to_adata(adata_target, vae)
    pre_velo = np.array(pre_u)*np.array(adata_target.var["fit_beta_velovi"]) - np.array(pre_s)*np.array(adata_target.var["fit_gamma_velovi"])
    corr = []
    for i in range(6):
        corr.append(scipy.stats.pearsonr(pre_velo[:,i],gt_velo[:,i])[0])
    
    ## print regvelo performance
    velo_velovi.append(np.mean(corr))
    print("veloVI: "+str(np.mean(corr)))
    
    print("Velocity: "+"RegVelo: "+str(velo_rgv[len(velo_rgv)-1])+" veloVI: "+str(velo_velovi[len(velo_velovi)-1])+" scVelo: "+str(velo_scv[len(velo_scv)-1]))

0


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 400/1500:  27%|██▋       | 400/1500 [01:06<03:03,  6.00it/s, loss=-8.89, v_num=1]  
Monitored metric elbo_validation did not improve in the last 45 records. Best score: -7.549. Signaling Trainer to stop.
RegVelo: 0.6191317085958523
recovering dynamics (using 1/112 cores)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


    finished (0:00:00) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)
computing velocities
    finished (0:00:00) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
veloVI: 0.5955228574695096
Epoch 318/500:  64%|██████▎   | 318/500 [00:38<00:21,  8.36it/s, loss=-7.26, v_num=1]
Monitored metric elbo_validation did not improve in the last 45 records. Best score: -7.040. Signaling Trainer to stop.
veloVI: 0.5739440557951783
Velocity: RegVelo: 0.6191317085958523 veloVI: 0.5739440557951783 scVelo: 0.5955228574695096
1


...

    finished (0:00:00) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)
computing velocities
    finished (0:00:00) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
veloVI: 0.7166638888386291
Epoch 325/500:  65%|██████▌   | 325/500 [00:28<00:15, 11.52it/s, loss=-9.53, v_num=1]
Monitored metric elbo_validation did not improve in the last 45 records. Best score: -8.954. Signaling Trainer to stop.
veloVI: 0.7136150639595148
Velocity: RegVelo: 0.7317771282385429 veloVI: 0.7136150639595148 scVelo: 0.7166638888386291
99


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 776/1500:  52%|█████▏    | 776/1500 [02:20<02:11,  5.51it/s, loss=-13.5, v_num=1]  
Monitored metric elbo_validation did not improve in the last 45 records. Best score: -12.943. Signaling Trainer to stop.
RegVelo: 0.5044344846644434
recovering dynamics (using 1/112 cores)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


    finished (0:00:00) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)
computing velocities
    finished (0:00:00) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
veloVI: 0.4406836885240834
Epoch 302/500:  60%|██████    | 302/500 [00:26<00:17, 11.21it/s, loss=-5.08, v_num=1]
Monitored metric elbo_validation did not improve in the last 45 records. Best score: -5.241. Signaling Trainer to stop.
veloVI: 0.4358963311292586
Velocity: RegVelo: 0.5044344846644434 veloVI: 0.4358963311292586 scVelo: 0.4406836885240834


In [12]:
dat = pd.DataFrame({"RegVelo":velo_rgv,"scVelo":velo_scv,"veloVI":velo_velovi})
if SAVE_DATASETS:
    dat.to_csv(DATA_DIR / 'simulation' / 'toy_GRN' / 'velocity_benchmark.csv')